In [1]:
from keras import applications
from keras.layers import Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import numpy as np
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [2]:
np.random.seed(1337)

#initial_model = applications.mobilenet.MobileNet(input_shape=(224,224, 3), alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=False, weights='imagenet', input_tensor=None, pooling='max', classes=5)

initial_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling='max', classes=5)


pred = Dense(5, activation = 'softmax')(initial_model.output)
model = Model(initial_model.input, pred)

model.compile(optimizer = Adam(lr = 1e-5), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 111, 111, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 111, 111, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 111, 111, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [ ]:
no_of_classes = 5
no_of_testing_images = 1566.0
no_of_training_images = 14591.0


trainingDataGenerator =  ImageDataGenerator(
    featurewise_center=True,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    zoom_range=0.2)


trainingDataGenerator = trainingDataGenerator.flow_from_directory('Data/train/spect/',class_mode='categorical',  batch_size = 16, target_size=(224,224))

validationDataGenerator = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=False, zca_whitening=False)

validationDataGenerator = validationDataGenerator.flow_from_directory('Data/test/spect/', class_mode='categorical',  batch_size = 16, target_size=(224,224))

checkpointer = ModelCheckpoint(filepath='weights_1.h5', verbose=1,save_best_only = True, monitor = 'val_accuracy')

tensorboard_callback = TensorBoard (log_dir = 'log/log_1', histogram_freq = 0, write_graph = True, write_images = False)



reduce_lr = ReduceLROnPlateau( #Reduce learning rate if loss doesn't improve
    monitor='val_accuracy',
    factor=0.1,
    patience=5,
    verbose=0,
    mode='min',
    epsilon=0.0001,
    cooldown=35,
    min_lr=1e-6)


fitGen = model.fit_generator(
    trainingDataGenerator,
    steps_per_epoch = no_of_training_images / 16,
    epochs = 100,
    validation_steps = no_of_testing_images / 16,
    validation_data = validationDataGenerator,
    verbose = 1,
    callbacks = [checkpointer, reduce_lr, tensorboard_callback]
)

Found 14591 images belonging to 5 classes.
Found 1566 images belonging to 5 classes.
Epoch 1/100


C:\Users\moham\Anaconda3\lib\site-packages\keras\preprocessing\image.py:506: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 31/911 [>.............................] - ETA: 17747s - loss: 3.0033 - acc: 0.2238

In [ ]:
plt.ylabel('Loss')
plt.xlabel('No. of epochs')
plt.plot(fitGen.history['loss'])
plt.plot(fitGen.history['val_loss'])
plt.show()